In [25]:
import msprime
from dismal import preprocess

In [76]:
def _coalescent_time_to_generations(t, N):
    return t*2*N

def _calculate_Ne(theta, mu):
    return theta/(4*mu)

In [110]:
# Parameter settings
theta0 = 5
theta1 = 5
theta2 = 5
theta1_prime = 5
theta2_prime = 5
t1 = 5
v = 5
m1_star = 0
m2_star = 0 
m1_prime_star = 0
m2_prime_star = 0 

mu_per_site = 1e-5
mu_per_generation = 1000
block_len = 100
num_replicates = 1000

In [111]:
demography = msprime.Demography()
demography.add_population(name="a", initial_size=_calculate_Ne(theta0, mu_per_generation))
demography.add_population(name="b1", initial_size=_calculate_Ne(theta1, mu_per_generation))
demography.add_population(name="b", initial_size=_calculate_Ne(theta2, mu_per_generation))
demography.add_population(name="c1", initial_size=_calculate_Ne(theta1_prime, mu_per_generation))
demography.add_population(name="c2", initial_size=_calculate_Ne(theta2_prime, mu_per_generation))

Population(initial_size=0.00125, growth_rate=0, name='c2', description='', extra_metadata={}, default_sampling_time=None, initially_active=None, id=4)

In [112]:
tau1 = t1/theta1
tau0 = v/theta1 + tau1
N = _calculate_Ne(theta1, mu_per_generation)
print(N)

0.00125


In [113]:
demography.add_population_split(time=_coalescent_time_to_generations((tau0), N), derived=["b1", "b"], ancestral="a")
demography.add_population_split(time=_coalescent_time_to_generations(tau1, N), derived=["c1"], ancestral="b1")
demography.add_population_split(time=_coalescent_time_to_generations(tau1, N), derived=["c2"], ancestral="b")
demography.set_migration_rate("b1", "b", m1_star)
demography.set_migration_rate("b", "b1", m2_star)
demography.set_migration_rate("c1", "c2", m1_prime_star)
demography.set_migration_rate("c2", "c1", m2_prime_star)
demography.sort_events()

In [114]:
ts_state1 = msprime.sim_ancestry(samples={'c1':2, 'c2':0}, demography=demography, sequence_length=block_len, num_replicates=num_replicates, ploidy=1)
ts_state2 = msprime.sim_ancestry(samples={'c1':0, 'c2':2}, demography=demography, sequence_length=block_len, num_replicates=num_replicates, ploidy=1)
ts_state3 = msprime.sim_ancestry(samples={'c1':1, 'c2':1}, demography=demography, sequence_length=block_len, num_replicates=num_replicates, ploidy=1)

In [116]:
s = []

s_dicts = []
for ts_state in [ts_state1, ts_state2, ts_state3]:
    s = []
    for ts in ts_state:
        mts = msprime.sim_mutations(ts, rate=mutation_rate, discrete_genome=False)
        s.append(mts.get_num_mutations())
    s_dicts.append(preprocess.counts_to_dict(s))
s_dicts

[{}, {0: 1000}, {0: 1000}]